In [1]:
import spiceypy
import datetime

In [2]:
datapath = '/home/einhard/Documents/projects/zData/spicepy/'

In [3]:
date_today = datetime.datetime.today()
date_today = date_today.strftime('%Y-%m-%dT00:00:00')

# Converting to ephemeris time
et_today_midnight = spiceypy.utc2et(date_today)

SpiceNOLEAPSECONDS: 
================================================================================

Toolkit version: CSPICE66

SPICE(NOLEAPSECONDS) --

The variable that points to the leapseconds (DELTET/DELTA_AT) could not be located in the kernel pool.  It is likely that the leapseconds kernel has not been loaded via the routine FURNSH.

utc2et_c --> UTC2ET --> TTRANS

================================================================================

The error above was thrown because *spicepy* needs to load the kernels for each mission, each session. They are not integrated with the spicepy library; instead, we can use the `spicepy.furnsh` function to load the required kernels from our hard drive:


In [4]:
spiceypy.furnsh(datapath + 'lsk/naif0012.tls')

In [5]:
et_today_midnight = spiceypy.utc2et(date_today)

In [6]:
print(f'The datetime {date_today} in ephemeris time is: {et_today_midnight}')

The datetime 2020-12-17T00:00:00 in ephemeris time is: 661435269.183502


To calculate the Earth's state vector we have to use the [NAIF IDs](https://naif.jpl.nasa.gov/pub/naif/toolkit_docs/C/req/naif_ids.html) to identify objects. 
Positive numbers are solar system bodiues, while negative numbers are spacecraft IDs. each instrument on a given spacecraft can be derived by multiplying the spacecraft ID by 1000 and then subtracting the ordinal number of the instrument.

$instrument = (Spacecraft ID * 1000) - InstrumentNo.$

The first 10 positive codes are used for Sun and planetary barycenters, i.e <br>
`0 = Solar System Barycenter`<br> `5 = Jupiter Barycenter`<br> etc. 

Planets and satellites are coded with a three digit number; the first number is its position in the solar system, while the last two refer to body's satellites - note that 99 is reserved for the planet itself:

`299 = Venus` <br> 
`301 = Moon` <br>
`399 = Earth`

In [7]:
# With the curent ET time in hand and the body ID known, we can calculate the current state of said body (the position and velocity). With the spkgeo function we can find the state vector and the light time of the body (light time is the travel time of light between the Sun and the target body). 

# Positions are in km, velocities in km/s, times in seconds.

# targ: the object we are interested in (NAIF ID)
# et: the ET of the computation
# ref: The reference frame. ECLIPJ2000 is the reference frame
# obs: The observer. That is, the center of the state vector calculation

EARTH_STATE_WRT_SUN, EARTH_SUN_LT = spiceypy.spkgeo(targ=399, \
                                                    et=ET_TODAY_MIDNIGHT, \
                                                    ref='ECLIPJ2000', \
                                                    obs=10)

NameError: name 'ET_TODAY_MIDNIGHT' is not defined

In [8]:
spiceypy.furnsh(datapath + 'spk/de432s.bsp')

In [9]:
EARTH_STATE_WRT_SUN, EARTH_SUN_LT = spiceypy.spkgeo(targ=399, \
                                                    et=et_today_midnight, \
                                                    ref='ECLIPJ2000', \
                                                    obs=10)

In [10]:
print(f'State vector of the Earth WRT the Sun for {date_today}: \n\n {EARTH_STATE_WRT_SUN}')

State vector of the Earth WRT the Sun for 2020-12-17T00:00:00: 

 [ 1.22713415e+07  1.46703924e+08 -6.67061039e+03 -3.01822425e+01
  2.36628969e+00  9.58957907e-04]


The state vector list is 6-dimensional: the first three values are x, y, z in km and their corresponding velocities in km/s

In [11]:
# Calculating Earth-Sun distance
import math
EARTH_SUN_DISTANCE = math.sqrt(EARTH_STATE_WRT_SUN[0]**2.0 \
                             + EARTH_STATE_WRT_SUN[1]**2.0 \
                             + EARTH_STATE_WRT_SUN[2]**2.0)

In [12]:
print(f'Earth-Sun distance in km: {EARTH_SUN_DISTANCE}')

Earth-Sun distance in km: 147216259.62814638


In [13]:
# To convert the km values in AU we can use the SPICE *convrt* function
EARTH_SUN_DISTANCE_AU = spiceypy.convrt(EARTH_SUN_DISTANCE, 'km', 'AU')
print(f'Earth-Sun distance in AU: {EARTH_SUN_DISTANCE_AU}')

Earth-Sun distance in AU: 0.9840799138666045


In [14]:
# Calculating orbital speed of Earth WRT Sun
EARTH_ORB_SPEED_WRT_SUN = math.sqrt(EARTH_STATE_WRT_SUN[3]**2.0 \
                                  + EARTH_STATE_WRT_SUN[4]**2.0 \
                                  + EARTH_STATE_WRT_SUN[5]**2.0)
print(f'Current orbial speed of Earth WRT Sun: {EARTH_ORB_SPEED_WRT_SUN} km/s')

Current orbial speed of Earth WRT Sun: 30.274859024388682 km/s


Another way to calculate orbital velocities is using this equation:

$ v_{orb} = sqrt(GM / r)$

where: <br><br>
    G = gravitational constant <br>
    M = mass of observing body <br>
    r = distance between bodies <br>
<br>

To find the required information we can load a PCK (Planetary Constants Kernel)

In [15]:
# With the function bodvcd we get the G*M value for the Sun (bodvcd doc). The bodyid is again 10, the required item is GM and maxn is an input parameter that sets the number of expected returns (here, it is 1):

# Loading kernel
spiceypy.furnsh(datapath + 'pck/gm_de431.tpc')
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

# Computing orbital speed
V_ORB_FUNC = lambda gm, r: math.sqrt(gm/r)
EARTH_ORD_SPEED_WRT_SUN_THEORY = V_ORB_FUNC(GM_SUN[0], EARTH_SUN_DISTANCE)

# Printing result
print(f'Theoretical orbital velocity of Earth WRT Sun: {EARTH_ORD_SPEED_WRT_SUN_THEORY} km/s')

Theoretical orbital velocity of Earth WRT Sun: 30.024648198769867 km/s
